In [ ]:
# mount collab to drive
from google.colab import drive
drive.mount("/content/drive")
%cd '/content/drive/My Drive/LlamaIndex/Structured data examples/SQL'

import pandas as pd
import numpy as np
!pip install llama_index
!pip install wikipedia
import os
import openai

openai.api_key = # your open AI key
from sqlalchemy import create_engine, MetaData, Table, Column, String, Integer, select, column
from llama_index import SQLDatabase, ServiceContext, SQLDatabase, ServiceContext, WikipediaReader, VectorStoreIndex
from llama_index.indices.struct_store.sql_query import NLSQLTableQueryEngine
from llama_index.llms import OpenAI
from sqlalchemy import (
    create_engine,
    MetaData,
    Table,
    Column,
    String,
    Integer,
    select,
    column,
    Float,
    text,
    insert
)
from llama_index.selectors.pydantic_selectors import (
    PydanticMultiSelector,
    PydanticSingleSelector,
)
from llama_index.tools.query_engine import QueryEngineTool
from llama_index.query_engine.router_query_engine import RouterQueryEngine
from llama_index.selectors.llm_selectors import LLMSingleSelector

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/LlamaIndex/Structured data examples/SQL


## Part I: Reading and inspecting the data

In [ ]:
# # https://www.socialexplorer.com/tables/C2020/R13427492
# # https://en.wikipedia.org/wiki/Greater_St._Louis
counties_data=pd.read_csv("greater_st_louis_housing_units.csv")
counties_data["vacant_unit_rates"]=((counties_data["vacant_units"]/counties_data["housing_units"]))
counties_data["vacant_unit_rates"]=counties_data["vacant_unit_rates"].astype(float)
counties_data["state"]=["MO" if str(i).startswith("29") else "IL" for i in list(counties_data["Geo_FIPS"])]

In [ ]:
counties_data

,Geo_FIPS,Geo_NAME,Geo_QNAME,housing_units,occupied_units,vacant_units,vacant_unit_rates,state
0,17005,Bond County,"Bond County, Illinois",6858,6164,694,0.101196,IL
1,17013,Calhoun County,"Calhoun County, Illinois",2282,1831,451,0.197634,IL
2,17027,Clinton County,"Clinton County, Illinois",15574,14258,1316,0.084500,IL
3,17083,Jersey County,"Jersey County, Illinois",9754,8652,1102,0.112979,IL
4,17117,Macoupin County,"Macoupin County, Illinois",21023,18804,2219,0.105551,IL
5,17119,Madison County,"Madison County, Illinois",118579,108607,9972,0.084096,IL
6,17133,Monroe County,"Monroe County, Illinois",14540,13752,788,0.054195,IL
7,17163,St. Clair County,"St. Clair County, Illinois",114595,103134,11461,0.100013,IL
8,29055,Crawford County,"Crawford County, Missouri",11378,9314,2064,0.181403,MO
9,29071,Franklin County,"Franklin County, Missouri",45275,41476,3799,0.083909,MO


# Part II: Demonstrating how to perform SQL queries using LLAMA-index

In [ ]:
# define the engine
engine = create_engine("sqlite:///:memory:")
metadata_obj = MetaData()

# create city SQL table
table_name = "city_stats"
city_stats_table = Table(
    table_name,
    metadata_obj,
    Column("county", String(16), primary_key=True),
    Column("vacant_unit_rate", Float),
    Column("State", String(2), nullable=False),
)
metadata_obj.create_all(engine)

# define the service index
llm = OpenAI(temperature=0.1, model="gpt-3.5-turbo")
service_context = ServiceContext.from_defaults(llm=llm)

# insert the data into the SQL database
sql_database = SQLDatabase(engine, include_tables=["city_stats"])
rows = []
for i in range(len(counties_data)):
  current_dict={"county":counties_data["Geo_NAME"][i],"vacant_unit_rate":counties_data["vacant_unit_rates"][i],"State":counties_data["state"][i]}
  rows.append(current_dict)

for row in rows:
    stmt = insert(city_stats_table).values(**row)
    with engine.connect() as connection:
        cursor = connection.execute(stmt)
        connection.commit()


# view current table
stmt = select(city_stats_table.c["county", "vacant_unit_rate", "State"]).select_from(
    city_stats_table
)
## check the table
with engine.connect() as connection:
    results = connection.execute(stmt).fetchall()
    print(results)
with engine.connect() as con:
    rows = con.execute(text("SELECT county from city_stats"))
    for row in rows:
        print(row)


# query the data
query_engine = NLSQLTableQueryEngine(
    sql_database=sql_database,
    tables=["city_stats"],
)

## perform the query
query_str = "Which county has the highest vacant unit rate?"
response = query_engine.query(query_str)

In [ ]:
print(response)

Calhoun County has the highest vacant unit rate with a rate of 0.19763365468886943.


## Part III: Showing how to query with both our SQL database + other unstructured data.

In [ ]:
# download wikipedia docs from each of these counties
wiki_docs = WikipediaReader().load_data(pages=list(counties_data["Geo_QNAME"]))

In [ ]:
# indexing each of these wikipedia descriptions of each county using the vector store index
counties=list(counties_data["Geo_QNAME"])
vector_indices = []
for wiki_doc in wiki_docs:
    vector_index = VectorStoreIndex.from_documents([wiki_doc])
    vector_indices.append(vector_index)

# define query engines for our structured + unstructured data seperately
vector_query_engines = [index.as_query_engine() for index in vector_indices]
sql_query_engine = NLSQLTableQueryEngine(
    sql_database=sql_database,
    tables=["city_stats"],
)

# preparing the structured data to be feed into RouterQueryEngine below
sql_tool = QueryEngineTool.from_defaults(
    query_engine=sql_query_engine,
    description=(
        "Useful for translating a natural language query into a SQL query over a table containing: "
        "city_stats, containing the vacant unit rates of counties in St Louis city"
    ),
)

# preparing the unstructured data to be feed into RouterQueryEngine below
vector_tools = []
for county, query_engine in zip(counties, vector_query_engines):
    vector_tool = QueryEngineTool.from_defaults(
        query_engine=query_engine,
        description=f"Useful for answering semantic questions about {county}",
    )
    vector_tools.append(vector_tool)

# here we combine the the information from the structured data (via sql_tool) and unstructured data (via vector_tools), such that we could query upon both these information.
query_engine = RouterQueryEngine(
    selector=PydanticSingleSelector.from_defaults(),
    query_engine_tools=([sql_tool] + vector_tools)
)



### Part 3A: Ask a question that is present in the unstructured data but not present in the structured data

In [ ]:
# query the data
response = query_engine.query("Which county has the highest African American demographic?")
print(str(response))

### part 3B: Ask a question that is in the structured data, but not in the unstructured data.

In [ ]:
response = query_engine.query("Which county has the highest vacant unit rate?")
print(str(response))

Calhoun County has the highest vacant unit rate with a rate of 0.19763365468886943.
